<a href="https://colab.research.google.com/github/Jinwooxxi/kagglestudy/blob/main/Kaggle%20survey/Kaggle_Survey_1st_kernel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

### Who are Data Seientists?

A data scientist ia a statistician or a programmer, who cleans, manages and organizes data, perform descriptive statistics and analysis to develop insights, build predictive models and sove business related problems.

In [ ]:
# !pip install squarify
# !pip install matplotlib-venn
# !pip install scipy==1.1.0
# !pip install plotly==2.7.0
# 아래 plotly 그래프는 다운그레이드 해야 가능

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import squarify
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import base64
import io
from scipy.misc import imread
import codecs
from IPython.display import HTML
from matplotlib_venn import venn2
from subprocess import check_output
print(check_output(["ls", "/content/drive/My Drive/kaggle_survey"]).decode("utf8"))

In [ ]:
response = pd.read_csv('/content/drive/My Drive/kaggle_survey/multipleChoiceResponses.csv',encoding='ISO-8859-1')
response.head()

### Some Basic Analysis

In [ ]:
print('The total number of respondents:', response.shape[0])
print('Total number of Countries with repondents:', response['Country'].nunique())
print('Counttry with highest respondens:', response['Country'].value_counts().index[0], 
      'with', response['Country'].value_counts().values[0], 'respondents')
print('Youngest respondents:', response['Age'].min(), ' and Oldest respondents', response['Age'].max())

### Gender Split

In [ ]:
plt.subplots(figsize=(22,12))
sns.countplot(y=response['GenderSelect'], order=response['GenderSelect'].value_counts().index)
plt.show()

### Respondent By Country

In [ ]:
resp_coun = response['Country'].value_counts()[:15].to_frame()
sns.barplot(resp_coun['Country'], resp_coun.index, palette='inferno')
plt.title('Top 15 Countries by number of respondents')
plt.xlabel('')
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.show()

tree = response['Country'].value_counts().to_frame()
squarify.plot(sizes=tree['Country'].values, label=tree.index, color=sns.color_palette('RdYlGn_r', 52))
plt.rcParams.update({'font.size':20})
fig = plt.gcf()
fig.set_size_inches(40,15)
plt.show()

### Compensation

Data Scientists are one of the most highest payed individuals.


In [ ]:
response['CompensationAmount'] = response['CompensationAmount'].str.replace(',','')
response['CompensationAmount'] = response['CompensationAmount'].str.replace('-','')

rates = pd.read_csv('/content/drive/My Drive/kaggle_survey/conversionRates.csv')
rates.drop('Unnamed: 0', axis=1, inplace=True)
salary = response[['CompensationAmount',
                 'CompensationCurrency',
                 'GenderSelect',
                 'Country',
                 'CurrentJobTitleSelect']].dropna()
salary = salary.merge(rates, left_on='CompensationCurrency', right_on='originCountry', how='left')
salary['Salary'] = pd.to_numeric(salary['CompensationAmount']) * salary['exchangeRate']

print('Maximun salary is USD $:', salary['Salary'].dropna().astype(int).max())
print('Minimun salary is USD $:', salary['Salary'].dropna().astype(int).min())
print('Median salary is USD $:', salary['Salary'].dropna().astype(int).median())

In [ ]:
plt.subplots(figsize=(15,8))
salary = salary[salary['Salary'] < 1000000]
sns.distplot(salary['Salary'])
plt.title('Salary Distribution', size=15)
plt.show()

#### Compensation by Country

In [ ]:
# 급여 높은 국가
f,ax = plt.subplots(1,2,figsize=(18,8))
sal_coun = salary.groupby('Country')['Salary'].median().sort_values(ascending=False)[:15].to_frame()
sns.barplot('Salary',sal_coun.index,data=sal_coun,palette='RdYlGn',ax=ax[0])
ax[0].axvline(salary['Salary'].median(), linestyle='dashed')
ax[0].set_title('Highest Salary Paying Countries')
ax[0].set_xlabel('')

# 응답 Top 15 국가들의 급여
max_coun = salary.groupby('Country')['Salary'].median().to_frame()
max_coun = max_coun[max_coun.index.isin(resp_coun.index)]
max_coun.sort_values(by='Salary', ascending=True).plot.barh(width=0.8, ax=ax[1], color=sns.color_palette('RdYlGn'))
ax[1].axvline(salary['Salary'].median(), linestyle='dashed')
ax[1].set_title('Compensation of Top 15 Respondent Countries')
ax[1].set_xlabel('')
ax[1].set_ylabel('')

plt.subplots_adjust(wspace=0.8)
plt.show()

#### Salary By Gender

In [ ]:
plt.subplots(figsize=(10,8))
sns.boxplot(y='GenderSelect', x='Salary', data=salary)
plt.ylabel('')
plt.show()

### Age

In [ ]:
plt.subplots(figsize=(15,8))
response['Age'].hist(bins=50, edgecolor='black')
plt.xticks(list(range(0,80,5)))
plt.title('Age Distribution')
plt.show() 

### Profession & Major

In [ ]:
f,ax = plt.subplots(1,2,figsize=(25,15))
sns.countplot(y=response['MajorSelect'], ax=ax[0], order=response['MajorSelect'].value_counts().index)
ax[0].set_title('Major')
ax[0].set_ylabel('')

sns.countplot(y=response['CurrentJobTitleSelect'], ax=ax[1], order=response['CurrentJobTitleSelect'].value_counts().index)
ax[1].set_title('Current Job')
ax[1].set_ylabel('')
plt.subplots_adjust(wspace=0.8)
plt.show()

Data Science and Machine Learning is used in almost every industry. This is evident from the left graph, as people from different arearas of interest like Physics, Biology, etc are taking it up for better understanding of the data. 

The rifht side graph shows the Current Job of the respondents. A major portion of the respondents are Data Scientists. But as it is survey data, we know that there may be many ambigious reponses. Later on we will check are these respondents real data-scientists or self proclaimed data-scientists.

### Compensation By Job Title

In [ ]:
sal_job = salary.groupby('CurrentJobTitleSelect')['Salary'].median().to_frame().sort_values(by='Salary',ascending=False)
ax = sns.barplot(sal_job.Salary, sal_job.index, palette=sns.color_palette('inferno',20))
plt.title('Compensation By Job Title', size=15)
for i, v in enumerate(sal_job.Salary): 
    ax.text(.5, i, v, fontsize=10, color='white',weight='bold')
fig = plt.gcf()
fig.set_size_inches(8,8)
plt.show()

Operations Research Practitioner has the highest median salary followed by Prdictive Modeler and Data Scientist. Coumputer Scientist and Programmers have the lowest compensation.

In [ ]:
f, ax = plt.subplots(1,2,figsize=(25,12))
skills = response['MLSkillsSelect'].str.split(',')
skills_set = []
for i in skills.dropna():
  skills_set.extend(i)
plt1 = pd.Series(skills_set).value_counts().sort_values(ascending=False).to_frame()
sns.barplot(plt1[0], plt1.index, ax=ax[0], palette=sns.color_palette('inferno_r', 15))
ax[0].set_title('ML Skills')

tech = response['MLTechniquesSelect'].str.split(',')
techniques = []
for i in tech.dropna():
  techniques.extend(i)
plt1 = pd.Series(techniques).value_counts().sort_values(ascending=False).to_frame()
sns.barplot(plt1[0], plt1.index, ax=ax[1], palette=sns.color_palette('inferno_r', 15))
ax[1].set_title('ML Techniques used')
plt.subplots_adjust(wspace=0.8)
plt.show()

In [ ]:
f,ax = plt.subplots(1,2,figsize=(25,12))
ml_nxt = response['MLMethodNextYearSelect'].str.split(',')
nxt_year = []
for i in ml_nxt.dropna():
    nxt_year.extend(i)
pd.Series(nxt_year).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9, color=sns.color_palette('winter_r', 15), ax=ax[0])

tool = response['MLToolNextYearSelect'].str.split(',')
tool_nxt = []
for i in tool.dropna():
    tool_nxt.extend(i)
pd.Series(tool_nxt).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9, color=sns.color_palette('winter_r', 15), ax=ax[1])

plt.subplots_adjust(wspace=0.8)
ax[0].set_title('ML Method Next Year')
ax[1].set_title('ML Tool Next Year')
plt.show()

### Best Platfomrs to Learn

In [ ]:
plt.subplots(figsize=(6,8))
learn = response['LearningPlatformSelect'].str.split(',')
platform = []
for i in learn.dropna():
    platform.extend(i)
pd.Series(platform).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9, color=sns.color_palette('winter',15))
plt.title('Best Platforms to Learn', size=15)
plt.show()

### Hardware Used

In [ ]:
plt.subplots(figsize=(10,10))
hard = response['HardwarePersonalProjectsSelect'].str.split(',')
hardware = []
for i in hard.dropna():
    hardware.extend(i)
pd.Series(hardware).value_counts().sort_values(ascending=True).plot.barh(width=0.9, color=sns.color_palette('inferno',10))
plt.title('Machines Used')
plt.show()

### Where Do I get Datasets From??

In [ ]:
plt.subplots(figsize=(15,15))
data = response['PublicDatasetsSelect'].str.split(',')
dataset = []
for i in data.dropna():
  dataset.extend(i)
pd.Series(dataset).value_counts().plot.pie(autopct='%1.1f%%', 
                                           shadow = True,
                                           colors=sns.color_palette('Paired', 10), 
                                           startangle=90,
                                           wedgeprops={'linewidth':2, 'edgecolor':'white'})
plt.title('Dataset Source')
my_circle = plt.Circle((0,0), 0.7, color='white')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.ylabel('')
plt.show()

### Code Sharing

In [ ]:
plt.subplots(figsize=(15,15))
code = response['WorkCodeSharing'].str.split(',')
code_share = []
for i in code.dropna():
    code_share.extend(i)
pd.Series(code_share).value_counts().plot.pie(autopct='%1.1f%%',
                                              shadow=True,
                                              colors=sns.color_palette('Set3',10),
                                              startangle=90,
                                              wedgeprops = {'linewidth':2, 'edgecolor':'white' })
plt.title('Code Sharing Medium')
my_circle = plt.Circle((0,0), 0.65, color='white')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.ylabel('')
plt.show()

### Challenges in Data Science

In [ ]:
plt.subplots(figsize=(15,18))
challenge = response['WorkChallengesSelect'].str.split(',')
challenges = []
for i in challenge.dropna():
    challenges.extend(i)
plt1 = pd.Series(challenges).value_counts().sort_values(ascending=False).to_frame()
sns.barplot(plt1[0],plt1.index, palette=sns.color_palette('inferno',25))
plt.title('Challenges in Data Science')
plt.show()

The main challenge in Data Science is getting the proper Data. The graph clearly shows that dirty data is the bigget challenge. Dirty data is a database record that contains errors. Dirty data can be caused by a number of factors including duplicate records, incomplete or outdated data, and the improper parsing of record fields from disparate systems. Luckily Kaggle datasets are pretty clean and standardised.

Some other major challenges are the Lack of Data Science and machine learning talent, difficulty in getting data and lack of tools. Thats why Data Science is the sexiest job in 21st century.With the increasing amount of data, this demand will substantially grow.

### Job Statisfaction

In [ ]:
satisfy = response.copy()
satisfy['JobSatisfaction'].replace({'10 - Highly Satisfied':'10',
                                    '1 - Highly Dissatisfied':'1',
                                    'I prefer not to share':np.NaN}, inplace=True)
satisfy.dropna(subset=['JobSatisfaction'], inplace=True)
satisfy['JobSatisfaction'] = satisfy['JobSatisfaction'].astype(int)
satisfy_job=satisfy.groupby(['CurrentJobTitleSelect'])['JobSatisfaction'].mean().sort_values(ascending=False).to_frame()
ax = sns.barplot(y=satisfy_job.index, x=satisfy_job.JobSatisfaction, palette=sns.color_palette('inferno',20))
fig = plt.gcf()
fig.set_size_inches(8,10)
for i, v in enumerate(satisfy_job.JobSatisfaction): 
    ax.text(.1, i, v,fontsize=10, color='white', weight='bold')
plt.title('Job Satisfaction out of 10')
plt.show()

#### Job Satisfication By Country

In [ ]:
satisfy = response.copy()
satisfy['JobSatisfaction'].replace({'10 - Highly Satisfied':10,
                                    '1 - Highly Dissatisfied':1,
                                    'I prefer not to share':np.NaN}, inplace=True)
satisfy.dropna(subset=['JobSatisfaction'], inplace=True)
satisfy['JobSatisfaction'] = satisfy['JobSatisfaction'].astype(int)
satisfy_job = satisfy.groupby(['Country'])['JobSatisfaction'].mean().sort_values(ascending=True).to_frame()

data = [dict(type='choropleth',
             autocolorscale=False,
             colorscale='Viridis',
             reversescale=True,
             locations=satisfy_job.index,
             z=satisfy_job['JobSatisfaction'],
             locationmode='country names',
             text=satisfy_job['JobSatisfaction'],
             marker=dict(line=dict(color='rgb(200,200,200)', width=0.5),
                         colorbar=dict(autotick=True, tickprefix='', title='Satisfaction')))]

layout = dict(
    title = 'Job Satisfaction By Country',
    geo = dict(
        showframe=True,
        showocean=True,
        projection=dict(
            type='chloropleth'
        ),
        lonaxis = dict(
            showgrid=False,
            gridcolor='rgb(102,102,102)'
        ),
        lataxis = dict(
            showgrid=False,
            gridcolor='rgb(102,102,102)'
        )
    )
)

fig = dict(data=data, layout=layout)
py.plot(fig, validate=False, filename='worldmap2010')
# py.iplot(fig, validate=False, filename='worldmap2010')

# Python vs R

Python and R are the most widely used Open-Source languages for Data Science and Machine-Learning stuff. For a budding data scientist or analyst, the biggest and trickiest doubt is: Which Language Should I Start With?? While both the languages have their own advantages and shortcomings, it depends on the individual's purpose while selecting a language of his/her choice. Both the languages cater the needs of different kinds of work. Python is a general purpose langauge, thus web and application integration is easier, while R is meant for pure statistical and analytics purpose. The area where R will completely beat Python is visualisations with the help of packages like ggplot2 and shiny. But Python has an upperhand in Machine Learning stuff

In [ ]:
resp = response.dropna(subset=['WorkToolsSelect'])
resp = resp.merge(rates, left_on='CompensationCurrency', right_on='originCountry', how='left')
python = resp[(resp['WorkToolsSelect'].str.contains('Python'))&(~resp['WorkToolsSelect'].str.contains('R'))]
R = resp[(~resp['WorkToolsSelect'].str.contains('Python'))&(resp['WorkToolsSelect'].str.contains('R'))]
both = resp[(resp['WorkToolsSelect'].str.contains('Python'))&(resp['WorkToolsSelect'].str.contains('R'))]

#### Recommended Language For Begineers

In [ ]:
response['LanguageRecommendationSelect'].value_counts()[:5].plot.bar()
plt.show()

###Recommendation By Python and R users

In [ ]:
labels1 = python['LanguageRecommendationSelect'].value_counts()[:5].index
sizes1 = python['LanguageRecommendationSelect'].value_counts()[:5].values

labels2 = R['LanguageRecommendationSelect'].value_counts()[:5].index
sizes2 = R['LanguageRecommendationSelect'].value_counts()[:5].values

fig = {
    'data' :[
             {
              'values' : sizes1,
              'labels' : labels1,
              'domain' : {'x':[0,.48]},
              'name' : 'Language',
              'hoverinfo' : 'label+percent+name',
              'hole' : .4,
              'type' : 'pie'
             },
             {
              'values' : sizes2,
              'labels' : labels2,
              'text' : 'CO2',
              'textposition' : 'inside',
              'domain' : {'x':[.54,1]},
              'name' : 'Language',
              'hoverinfo' : 'label+percent+name',
              'hole' : .4,
              'type' : 'pie'
             }
             ],
    'layout' : {
        'title' : "Language Recommend By Python and R Users",
        'annotations' : [
                         {
                          'font': {'size':30},
                          'showarrow' : False,
                          'text' : 'Python',
                          'x' : 0.17,
                          'y' : 0.5
                         },
                         {
                          'font': {'size':30},
                          'showarrow' : False,
                          'text' : 'R',
                          'x' : 0.79,
                          'y' : 0.5
                         }]
    }
}

# py.iplot(fig, filename='donut')
py.plot(fig, filename='donut')

This is a interesting find. About 91.6% Python users recommend Python as the first language for begineers, whereas only 67.2% R users recommend R as the first language. Also 20.6% R users recommend Python but only 1.68% Python users recommend R as the first language. One thing to note is that users of both recommend the same Languages i.e SQL, Matlab and C/C++. I have only considered the Top 5 recommended languages, so the percentage will change if we consider all of them. But the difference would be just 2-3%.

#### Necessary or Not?

In [ ]:
f, ax = plt.subplots(1,2,figsize=(18,8))
response['JobSkillImportancePython'].value_counts().plot.pie(ax=ax[0], autopct='%1.1f%%', explode=[0.1,0,0],
                                                             shadow=True, colors=['g','lightblue','r'])
ax[0].set_title('Python Necessary')
ax[0].set_ylabel('')

response['JobSkillImportanceR'].value_counts().plot.pie(ax=ax[1], autopct='%1.1f%%', explode=[0.1,0,0],
                                                        shadow=True, colors=['g','lightblue','r'])
ax[1].set_title('R Necessary')
ax[1].set_ylabel('')
plt.show()

#### Number Of Users By Language

In [ ]:
f, ax = plt.subplots(1,2,figsize=(18,8))
pd.Series([python.shape[0], R.shape[0], both.shape[0]], index=['Python', 'R', 'Both']).plot.bar(ax=ax[0])
ax[0].set_title('Number of Users')

venn2(subsets = (python.shape[0], R.shape[0], both.shape[0]), set_labels=('Python Users', 'R Users'))
plt.title('Venn Diagram for Users')
plt.show()

The number of Python users are definetely more than R users. This may be due to the easy learning curve of Python. However there are more users who know both the languages. These responses might be from established Data Scientists,as they tend to have a knowledge in multiple languages and tools.

### Compensation

In [ ]:
py_sal = (pd.to_numeric(python['CompensationAmount'].dropna()) * python['exchangeRate']).dropna()
py_sal = py_sal[py_sal < 1000000]

R_sal = (pd.to_numeric(R['CompensationAmount'].dropna()) * R['exchangeRate']).dropna()
R_sal = R_sal[R_sal < 1000000]

both_sal = (pd.to_numeric(both['CompensationAmount'].dropna()) * both['exchangeRate']).dropna()
both_sal = both_sal[both_sal < 1000000]

trying = pd.DataFrame([py_sal, R_sal, both_sal])
trying = trying.T
trying.columns = ['Python', 'R', 'Both']

print('Median Salary For Individual using Python', trying['Python'].median())
print('Median Salary For Individual using R', trying['R'].median())
print('Median Salary For Individual knowing both language', trying['Both'].median())

In [ ]:
trying.plot.box()
plt.title('Compensation By Language')
fig = plt.gcf()
fig.set_size_inches(10,6)
plt.show()

### Language Used By Professionals

In [ ]:
py1 = python.copy()
r = R.copy()
py1['WorkToolsSelect'] = 'Python'
r['WorkToolsSelect'] = 'R'
r_vs_py = pd.concat([py1,r])
r_vs_py = r_vs_py.groupby(['CurrentJobTitleSelect','WorkToolsSelect'])['Age'].count().to_frame().reset_index()
r_vs_py.pivot('CurrentJobTitleSelect','WorkToolsSelect','Age').plot.barh(width=0.8)
fig = plt.gcf()
fig.set_size_inches(10,15)
plt.title('Job Title vs Language Used',size=15)
plt.show()

### Job Function vs Language

In [ ]:
r_vs_py = pd.concat([py1, r])
r_vs_py = r_vs_py.groupby(['JobFunctionSelect', 'WorkToolsSelect'])['Age'].count().to_frame().reset_index()
r_vs_py.pivot('JobFunctionSelect', 'WorkToolsSelect', 'Age').plot.barh(width=0.8)
fig = plt.gcf()
fig.set_size_inches(10,15)
plt.title('Job Description vs Language Used')
plt.show()

### Tenure vs Language Used

In [ ]:
r_vs_py = pd.concat([py1, r])
r_vs_py = r_vs_py.groupby(['Tenure', 'WorkToolsSelect'])['Age'].count().to_frame().reset_index()
r_vs_py.pivot('Tenure', 'WorkToolsSelect', 'Age').plot.barh(width=0.8)
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.title('Job Tenure vs Language Used')
plt.show()

### Industry vs Language Used

In [ ]:
r_vs_py = pd.concat([py1, r])
r_vs_py = r_vs_py.groupby(['EmployerIndustry', 'WorkToolsSelect'])['Age'].count().to_frame().reset_index()
r_vs_py.pivot('EmployerIndustry', 'WorkToolsSelect', 'Age').plot.barh(width=0.8)
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.title('Industry vs Language Used')
plt.show()

### Common Tools with Python and R

In [ ]:
f, ax = plt.subplots(1,2,figsize=(20,15))
py_comp = python['WorkToolsSelect'].str.split(',')
py_comp1 = []
for i in py_comp:
  py_comp1.extend(i)
plt1 = pd.Series(py_comp1).value_counts()[1:15].sort_values(ascending=False).to_frame()
sns.barplot(plt1[0], plt1.index, ax=ax[0], palette=sns.color_palette('flare', 15))

R_comp = R['WorkToolsSelect'].str.split(',')
R_comp1 = []
for i in R_comp:
  R_comp1.extend(i)
plt1 = pd.Series(R_comp1).value_counts()[1:15].sort_values(ascending=False).to_frame()
sns.barplot(plt1[0], plt1.index, ax=ax[1], palette=sns.color_palette('flare', 15))

ax[0].set_title('Commonly Used Tools with Python')
ax[1].set_title('Commonly Used Tools with R')

plt.subplots_adjust(wspace=0.8)
plt.show()

# Asking the Data Scientists

In [ ]:
response['DataScienceIdentitySelect'].value_counts()

In [ ]:
plt.subplots(figsize=(10,8))
scientist = response[response['DataScienceIdentitySelect'] == 'Yes']
scientist['CurrentJobTitleSelect'].value_counts().sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('light:#5A9',15))
plt.title('Job Titles', size=15)
plt.show()

Surprisingly there is no entry for the Job Title Data Scientist. There reasons for this could be that the people with CurrentJobTitleSelect as Data Scientist(who might be working as Data Scientist) might have not answered the question: "Do you currently consider yourself a Data Scientist?"

There are many overlapping and common skills between the jobs like Data Analyst,Data Scientist and Machine Learning experts, Statisticians,etc. Thus they too have similar skills and consider themselves as Data Scientists even though they are not labeled the same. Now lets check if the previous assumption was True.

In [ ]:
true = response[response['CurrentJobTitleSelect'] == 'Data Scientist']

It was indeed True. People with their CurrentJobTitle as Data Scientist did not answer the question "Do you currently consider yourself a Data Scientist?". So I am considering them also to be real Data Scientists.

In [ ]:
scientist = pd.concat([scientist, true])
scientist['CurrentEmployerType'].shape[0]

So out of the total respondents, about 40% of them are Data Scientists or have skills for the same.



### Country-Wise Split

In [ ]:
plt.subplots(figsize=(10,8))
coun = scientist['Country'].value_counts()[:15].sort_values(ascending=False).to_frame()
sns.barplot(coun.Country, coun.index, palette='inferno')
plt.title('Countries By Number Of data Scientists', size=15)
plt.show()

The graph is similar to the demographic graph where we had shown number of users by country. The difference is that the numbers have reduced as we have only considered Data Scientists.



### Employment Status & Education

In [ ]:
f, ax = plt.subplots(1,2,figsize=(25,10))
sns.countplot(y=scientist['EmploymentStatus'], ax=ax[0])
ax[0].set_title('Employment Status')
ax[0].set_ylabel('')

sns.countplot(y=scientist['FormalEducation'], 
              order=scientist['FormalEducation'].value_counts().index, 
              ax=ax[1], palette=sns.color_palette('viridis_r', 15))
ax[1].set_title('Formal Education')
ax[1].set_ylabel('')

plt.subplots_adjust(wspace=0.5)
plt.show()

About **67%** of the data scientists are employed full-time, while about **11-12%** of them are unemployed but looking for job. On the education side it is evident that about 45-46% of the data scientists hold a **master's degree**, while about **23-24%** of them have **a bachelor's degree or a doctoral degree**. Thus education seems to be an important factor for becoming a data scientist. Let's see how does the salary vary according to the education.



### Compensation By Formal Education

In [ ]:
plt.subplots(figsize=(25,12))
comp_edu = scientist.merge(salary, left_index=True, right_index=True, how='left')
comp_edu = comp_edu[['FormalEducation','Salary']]
sns.boxplot(x='FormalEducation', y='Salary', data=comp_edu)
plt.title('Compensation vs Education')
plt.xticks(rotation=90)
plt.show()

This is surprising as the salary ranges for Bachelor's, Master's and Doctoral degree look to very similar. The median for Bachelor's degree seems to be a bit high as compared to Master's and doctoral degree. I didn't expect this as many of the Data Scientists hold a masters degree. But I think Work Experience matters more than any degree. Maybe the Bachelor's degree holders have more experience as compared than the other two.

### Previous Job and Salary Change

In [ ]:
f,ax = plt.subplots(1,2,figsize=(30,15))
past = scientist['PastJobTitlesSelect'].str.split(',')
past_job = []
for i in past.dropna():
    past_job.extend(i)
pd.Series(past_job).value_counts().sort_values(ascending=True).plot.barh(width=0.9, color=sns.color_palette('summer',25),ax=ax[0])
ax[0].set_title('Previous Job')

sal = scientist['SalaryChange'].str.split(',')
sal_change = []
for i in sal.dropna():
    sal_change.extend(i)
pd.Series(sal_change).value_counts().sort_values(ascending=True).plot.barh(width=0.9, color=sns.color_palette('summer',10),ax=ax[1])
ax[1].set_title('Salary Change')
plt.subplots_adjust(wspace=0.9)
plt.show()

Clearly majority of people switching to Data Science get a salary hike about 6-20% or more. With this increasing demand for Data Scientists, the salary may also increase with time.



### Tools used at Work

In [ ]:
plt.subplots(figsize=(8,8))
tools = scientist['WorkToolsSelect'].str.split(',')
tools_work = []
for i in tools.dropna():
  tools_work.extend(i)
pd.Series(tools_work).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9, color=sns.color_palette('RdYlGn', 15))
plt.show()

### Where Did they Learn From?

In [ ]:
course = scientist['CoursePlatformSelect'].str.split(',')
course_plat = []
for i in course.dropna():
  course_plat.extend(i)
course_plat = pd.Series(course_plat).value_counts()

blogs = scientist['BlogsPodcastsNewslettersSelect'].str.split(',')
blogs_fam = []
for i in blogs.dropna():
  blogs_fam.extend(i)
blogs_fam = pd.Series(blogs_fam).value_counts()

labels1 = course_plat.index
sizes1 = course_plat.values

labels2 = blogs_fam[:5].index
sizes2 = blogs_fam[:5].values

fig = {
    'data' : [
              {
               'values':sizes1,
               'labels':labels1,
               'domain': {'x':[0,.48]},
               'name':'MOOC',
               'hoverinfo':'label+percent+name',
               'hole':.4,
               'type':'pie'
              },
              {
               'values':sizes2,
               'labels':labels2,
               'text':'CO2',
               'textposition':'inside',
               'domain': {'x':[.54,1]},
               'name':'MOOC',
               'hoverinfo':'label+percent+name',
               'hole':.4,
               'type':'pie'
              }
    ],
    'layout' : {
        'title' : 'Blogs and Oline Platforms',
        'showlegend' : True,
        'annotations': [
                        {
                         'font': {'size':12},
                         'showarrow' : False,
                         'text':'MOOC s',
                         'x':0.18,
                         'y':0.5
                        },
                        {'font':{'size':12},
                         'showarrow':False,
                         'text':'BLOGS',
                         'x':0.83,
                         'y':0.5
                         }
        ]
    }
}

py.plot(fig, filename='donut2')

Coursera is the most favoured platform by Data Scientists for learning Data Science. My personal vote also goes for Coursera, where you can learn things from scratch to advanced on the same platform. It is not limited to a single language like Python or R, but also has courses covering other languages like Scala,etc. Similarly KDNuggets is the most preferred blog.



### Time Spent on Tasks

A Data Scientist is not always building predictive models, he is also responsible for the data quality, gathering the right data, analytics, etc.

In [ ]:
import itertools

plt.subplots(figsize=(22,10))
time_spent = ['TimeFindingInsights', 'TimeVisualizing', 'TimeGatheringData', 'TimeModelBuilding']
length = len(time_spent)

for i, j in itertools.zip_longest(time_spent, range(length)):
  plt.subplot((length/2),2,j+1)
  plt.subplots_adjust(wspace=0.2, hspace=0.5)
  scientist[i].hist(bins=10, edgecolor='black')
  plt.axvline(scientist[i].mean(), linestyle='dashed', color='r')
  plt.title(i, size=20)
  plt.xlabel('% Time')

plt.show()

* **TimeGatheringData**: It is undoubtedly the most time consuming part. Getting the data is the most painstaking task inthe entire process, which is followed by Data Cleaning(not shown as data not available) which is yet other time consuming process. Thus gathering right data and scrubing the data are the most time consuming process.

* **TimeVisualizing**: It is probably the least time consuming process(and probably the most enjoyable one..:p), and it reduces even further if we use Enterprise Tools like Tableau,Qlik,Tibco,etc, which helps in building graphs and dashboards with simple drag and drop features.

* **TimeFindingInsights**: It is followed after visualising the data, which involves finding facts and patterns in the data, slicing and dicing it to find insights for business processes.It looks to a bit more time consuming as compared to TimeVisualizing.

* **TimeModelBuilding**: It is where the data scientists build predictive models, tune these models,etc. It is the 2nd most time consuming process after TimeDataGathering.

### Cloud Servies

With the increasing size of the data, it is not possible to process the data and perform predictive anaytics on the physical srver infrastructures. Cloud thus takes predictive analyics to a next level, with scalability its main advantage. They manage srvice that enables you to easily build machine learning model that work on any type of data, of any size. 

In [ ]:
cloud = ['WorkToolsFrequencyAmazonML', 'WorkToolsFrequencyAWS', 'WorkToolsFrequencyCloudera',
         'WorkToolsFrequencyHadoop', 'WorkToolsFrequencyAzure']

plt.subplots(figsize=(30,15))
length = len(cloud)

for i, j in itertools.zip_longest(cloud, range(length)):
  plt.subplot((length/2+1), 3, j+1)
  plt.subplots_adjust(wspace=0.2, hspace=0.5)
  sns.countplot(i, data=scientist)
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')

plt.show()

It is evident that AmazonAWS, which is a public cloud service provider is the most used cloud platform, followed by Hadoop. Hadoop is an open-source software framework used for distribued storage and processing of dataset of big data.

### Importance Of Visualisations

In [ ]:
f, ax = plt.subplots(1,2, figsize=(25,12))

sns.countplot(scientist['JobSkillImportanceVisualizations'], ax=ax[0])
ax[0].set_title('Job Importance For Visuals')
ax[0].set_xlabel('')

scientist['WorkDataVisualizations'].value_counts().plot.pie(autopct='%2.0f%%', colors=sns.color_palette('Paired', 10), ax=ax[1])
ax[1].set_title('Use Of Visualizations in Projects')
my_circle = plt.Circle((0,0), 0.7, color='white')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.ylabel('')

plt.show()

Visualizations ar a very integral part of Data Science Projects, and the above graph also show the same. Almost all data science project i.e99% of the projects have visualizations in them, doesn't matter how big or small. About 95% of data Scientists say that Visualizations skills are nice to have or necessary. Visuals help to understand and comprehend the data fater not only to the professionals but also to target customers, who may not be technically skilled.

### BI Tools

Business intelligence software is a type of application software designed to retrivev, analyze, transform and report data for buness intelligence. They make data visualization and analytics vety imple as compared to normal coding way in Python or R. The only drawback is that they are proprietory and costly. Lets check wich are the most frequently used enterprise BI tolls used by Data Scientists.

In [ ]:
BI = ['WorkToolsFrequencyQlik','WorkToolsFrequencySAPBusinessObjects',
      'WorkToolsFrequencyTableau','WorkToolsFrequencyTIBCO',
      'WorkToolsFrequencyAngoss','WorkToolsFrequencyIBMCognos',
      'WorkToolsFrequencyKNIMECommercial','WorkToolsFrequencyExcel']
   
plt. subplots(figsize=(30,25))
length = len(BI)

for i, j in itertools.zip_longest(BI, range(length)):
  plt.subplot((length/2+1), 3, j+1)
  plt.subplots_adjust(wspace=0.2, hspace=0.5)
  sns.countplot(i, data=scientist)
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')

plt.show()

Excel is the bread and butter for analysts. This thing still somewhat holds true, as Excel is still popular among Data Scientists. However, the most frequently used BI tool looks to be Tableau.

### Knowledge Of Algorithms (Maths and Stats)

In [ ]:
f,ax = plt.subplots(1,2,figsize=(25,12))
sns.countplot(y=scientist['AlgorithmUnderstandingLevel'],
              order=scientist['AlgorithmUnderstandingLevel'].value_counts().index,
              ax=ax[0],palette=sns.color_palette('summer',15))
sns.countplot(scientist['JobSkillImportanceStats'], ax=ax[1])
ax[0].set_title('Algorithm Understanding')
ax[0].set_ylabel('')
ax[1].set_title('Knowledge of Stats')
ax[1].set_xlabel('')
plt.show()

Data Scientists have a good knowledge of mathematical concepts like Statistics and Linear Algebra, which are the most important part of Machine Learning algorithms. But is this maths really required, as many standard libraries like scikit,tensorflow,keras etc have all these things already implemented. But the experienced data scientists say that we should have a good understanding of the maths behind the algorithms. About 95% of the data scientists say the stats is an important asset in Data Science.

### Learning Platform Usefullness

In [ ]:
plt.subplots(figsize=(25,35))
useful = ['LearningPlatformUsefulnessBlogs', 'LearningPlatformUsefulnessCollege',
          'LearningPlatformUsefulnessCompany','LearningPlatformUsefulnessKaggle', 
          'LearningPlatformUsefulnessCourses','LearningPlatformUsefulnessProjects', 
          'LearningPlatformUsefulnessTextbook','LearningPlatformUsefulnessYouTube']
length = len(useful)

for i, j in itertools.zip_longest(useful, range(length)):
  plt.subplot((length/2), 3, j+1)
  plt.subplots_adjust(wspace=0.2, hspace=0.2)
  scientist[i].value_counts().plot.pie(autopct='%1.1f%%', colors=['g', 'lightblue', 'r'],
                                       wedgeprops={'linewidth':2, 'edgecolor':'white'})
  plt.title(i, size=25)
  my_circle = plt.Circle((0,0), 0.7, color='white')
  p = plt.gcf()
  p.gca().add_artist(my_circle)
  plt.xlabel('')
  plt.ylabel('')

plt.show()

The above dount charts shows the opinion of Data Scientists about the various platforms to learn Data Science. The plot looks best for Projects, where the percentage for not useful is almost 0%. Project are the best platform or way for learning anything in the IT industry. The other excellent platforms are Online Courses and Kaggel. The graphs for other platforms are quit similar to each other.

What should the Resume Have?

In [ ]:
f, ax = plt.subplots(1,2, figsize=(22,8))
sns.countplot(y=scientist['ProveKnowledgeSelect'],
              order=scientist['ProveKnowledgeSelect'].value_counts().index,
              ax=ax[0], palette=sns.color_palette('inferno',15))
ax[0].set_title('How to prove my knowledge')
sns.countplot(scientist['JobSkillImportanceKaggleRanking'],ax=ax[1])
ax[1].set_title('Kaggle Rank')
plt.show()

It is evident that Work experience in ML projects and Kaggle competitions reflects the knowledge of Data Science. Also a kaggle rank can be a good thing in one's resume. As I had mentioned earlier that relevant work experience might have a higher value as compared to any Master's or Doctoral degree. This statement thus holds good, as Data Scientists prefer work experience over degree, as seen in the above graph.

### How did they search for Jobs?

In [ ]:
plt.subplots(figsize=(10,8))
scientist.groupby(['EmployerSearchMethod'])['Age'].count().sort_values(ascending=True).plot.barh(width=0.8, color=sns.color_palette('winter',10))
plt.title('Job Search Method',size=15)
plt.ylabel('')
plt.show()

### Checking the Free Responses

This file contains the free form responses answered by the repondents. The problem with this one is that being a free form response, every user will answer in their own way. An example of this that I observed is the library Pandas is written as pandas, Pandas, panda and in many such differnt forms. Thus I will try to analyse this file using nltk(Natural Language Toolkit).

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

free = pd.read_csv('/content/drive/My Drive/kaggle_survey/freeformResponses.csv')
stop_words = set(stopwords.words('english'))
stop_words.update(',',';','!','?','.','(',')','$','#','+',':','...')

### Motivation Behind Working on Kaggle

In [ ]:
kaggle=b'...'

In [ ]:
motivation = free['KaggleMotivationFreeForm'].dropna().apply(nltk.word_tokenize)
motivate = []
for i in motivation:
  motivate.extend(i)
motivate = pd.Series(motivate)
motivate = ([i for i in motivate.str.lower() if i not in stop_words])

f1 = open('kaggle.png', 'wb')
f1.write(codecs.decode(kaggle, 'base64'))
f1.close()

img1 = imread('kaggle.png')
hcmask1 = img1

wc = WordCloud(background_color='black', max_words=4000, mask=hcmask1, stopwords=STOPWORDS, 
               max_font_size=60, width=1000, height=1000)
wc.generate(' '.join(motivate))
plt.imshow(wc)
plt.axis('off')
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.show()

### Most Frequently Used Libraries

In [ ]:
library = free['WorkLibrariesFreeForm'].dropna().apply(nltk.word_tokenize)
lib = []
for i in library:
  lib.extend(i)

lib = pd.Series(lib)
lib = ([i for i in lib.str.lower() if i not in stop_words])
lib = pd.Series(lib)
lib = lib.value_counts().reset_index()
lib.loc[lib['index'].str.contains('Pandas|pandas|panda'), 'index'] = 'Pandas'
lib.loc[lib['index'].str.contains('Tensorflow|tensorflow|tf|tensor'), 'index'] = 'Tensorflow'
lib.loc[lib['index'].str.contains('Scikit|scikit|sklearn'), 'index'] = 'Sklearn'
lib = lib.groupby('index')[0].sum().sort_values(ascending=False).to_frame()

R_packages = ['dplyr', 'tidyr', 'ggplot2', 'caret', 'randomforest', 'shiny', 'R markdown',
              'ggmap', 'leaflet', 'ggvis', 'stringr', 'tidyverse', 'plotly']
Py_packages = ['Pandas','Tensorflow','Sklearn','matplotlib','numpy','scipy','seaborn',
               'keras','xgboost','nltk','plotly']

f, ax = plt.subplots(1,2, figsize=(18,10))
lib[lib.index.isin(Py_packages)].sort_values(by=0, ascending=True).plot.barh(ax=ax[0], width=0.9, color=sns.color_palette('viridis', 15))
ax[0].set_title('Most Frequently Used Py Libraries')

lib[lib.index.isin(R_packages)].sort_values(by=0, ascending=True).plot.barh(ax=ax[1], width=0.9, color=sns.color_palette('viridis', 15))
ax[1].set_title('Most Frequently Used R Libraries')

ax[1].set_ylabel('')
plt.show()

**Python:**

1) Sklearn- For Machine Learning algorithms. This library has almost all the important machine learning algorithms used for industries.

2) Pandas, Matlotlib & Seaborn- Mostly used together for analytics and visualisation work.

3) TensorFlow and Keras- Used for Deep Learning.

4) Numpy and Scipy- Used for scientific computations.

5) nltk- Used for Natural Language Processing.

**R:**

1) dplyr- dplyr is the package for fast data manipulation.

2) ggplot2 and shiny- R's famous package for making beautiful graphics. Python visuals are nowhere near to the quality of visuals created using this library.

3) caret and randomforest- For Machine Learning purpose.

4) tidyr- Tools for changing the layout of your data sets.

5) stringr- Easy to learn tools for regular expressions and character strings.

Leaflet (folium in Python) and Plotly are common libraries in both languages, and are used to create interactive plots like geomaps, etc.

# Conclusions

1. Majority of the repondents are from USA followed by India. USA also hade the maximum number of data scientitst followed by India. Also the median Salary is highest in USA.

2. Majority of the respondents are in the age bracket 20-35, which shows that data science is quite famous in the youngsters.

3. The respondents are not just limited to Computer Science major, but also from majors like Statistics, health sciences,etc showing that Data Science is an interdisciplinary domain.

4. Majority of the respondents are fully employed.

5. Kaggle, Online Courses(Coursera,eDx,etc), Projects and Blogs(KDNuggets,AnalyticsVidya,etc) are the top resources/platforms for learning Data Science.

6. Kaggle has the highest share for data acquisition whereas Github has the highest share for code sharing.

7. Data Scientists have the highest Job Satisfaction level and the second highest median salary (after Operations Research Analyst). On the contrary, Programmers have the least Job Satisfaction level and one of the least median salary also.

8. Data Scientists also get a hike of about 6-20% from their previous jobs.

Tips For Budding Data Scientists

1. Learn Python, R and SQL as they are the most used languages by the Data Scientists. Python and R will help in analytics and predictive modeling while SQL is best for querying the databases.

2. Learn Machine Learning Techniques like Logistic Regression, Decision Trees, Support Vector Machines, etc as they are most commonly used Machine Learning techniques/algorithms.

3. Deep Learning and Neural Nets will be the most sought after techniques in the future, thus a good knowledge in them will be very helpful.

4. Develop skills for Gathering Data and Cleaning The Data as they are the most time consuming processes in the workflow of a data scientist.

5. Visualisations are very important in Data Science projects and almost all projects require Visualisations for understanding the data better. So one should learn Data Visualisation as Data Scientists consider it to be a necessary or nice to have skill.

6. Maths and Stats are very important in Data Science, so we should have good understanding of it for actually understanding how the algorithm works.

7. Projects are the best way to learn Data Science according to Data Scientists.So working on projects will help you learn data science better.

8. Experience with ML Projects in company and Kaggle Competitions are the best ways to show your working knowledge in Data Science. Working on ML projects in a company gives the experience of working with real world datasets, thereby enhancing the knowledge. Kaggle competitions are also a great medium, as you will be competing with Data Scientists over the world. Also a Kaggle Rank can be a good USP in the resume.